# API OpenWeatherMap



## Configuration de l'application:

- api_key

Définition des fonctions pour la gestion de la configuration

In [5]:
# Configuration de base

import json
import os
import requests

# verifie l'existence du fichier config.json
def exist_config_file():
    existe = False

    files = os.listdir('.')
    if 'config.json' in files:
        existe = True

    return( existe )

# Lecture de la configuration dans config.json
def read_config_file():
    if exist_config_file():
        with open('./config.json') as fichier:
            print("Chargement configuration")
            config = json.load(fichier)

    return( config )

# Ecriture de la config config.json
def write_config_file( config ):
    with open('./config.json', 'w') as fichier:
        json.dump(config, fichier)

    return

def get_config( param ):
    val = ""
    if param in config:
        val = config[param]

    return( val )

def set_config( param, val ):
    config[param] = val

    return( True )

def del_config( param ):
    del config[param]
    
    return( True )

### Chargement de la configuration sinon création d"une configuration de base

In [34]:
# Chargement de la configuration ou création d"une configuration de base

# Variable de configuration globale
config = {}

if exist_config_file():
    
    config = read_config_file()
    print(config)
    
else:
    print("Erreur pas de fichier de configuration config.json")
    
    api_key = input("Entrez votre clé d'API openWeather Map: ")
    set_config( "api_key", api_key )
    
    write_config_file( config )
    

Chargement configuration
{'api_key': 'a796841ff7a6a3e25e468f5f06628209', 'lat': 45.4401467, 'lon': 4.3873058}


### Geocoding API

Documentation : https://openweathermap.org/api/geocoding-api

Faire une requete pour rechercher une ville:

In [32]:
ville = input("Saisir une ville : ")
pays = input("Code pays (ISO 3166) : ")

url_api = "http://api.openweathermap.org/geo/1.0/direct?limit=20&appid=" + get_config("api_key")
url_api += "&q=" + ville
if len(pays) > 0:
    url_api += "," + pays

# Executer la requete
req = requests.get( url_api )
api_data = req.json()

# print(api_data)

geo_match = []

for index_geo, geo_loc in enumerate(api_data):
    local_name = ""
    if "local_names" in geo_loc:
            for key in geo_loc["local_names"]:
                if local_name == "" and key == "fr":
                    local_name = geo_loc["local_names"][key]
    geo_match.append( [geo_loc["name"], local_name, geo_loc["lat"], geo_loc["lon"], geo_loc["country"], geo_loc["state"] ] )

# Affichage des localisations trouvées
for index_geo, geo_loc in enumerate(geo_match):
    print(index_geo, geo_loc)
    

Saisir une ville : saint etienne
Code pays (ISO 3166) : 
0 ['Saint-Étienne', 'Saint-Étienne', 45.4401467, 4.3873058, 'FR', 'Auvergne-Rhône-Alpes']
1 ['Saint-Étienne-la-Cigogne', 'Saint-Étienne-la-Cigogne', 46.1148482, -0.5034125, 'FR', 'Nouvelle-Aquitaine']
2 ['Saint-Étienne', 'Saint-Étienne', 45.409604200000004, 4.484915566669622, 'FR', 'Auvergne-Rhône-Alpes']
3 ['Saint Etienne', '', -29.1458205, -51.1529959, 'BR', 'Rio Grande do Sul']
4 ['Saint-Etienne', '', 44.6995545, 4.8478246, 'FR', 'Auvergne-Rhône-Alpes']


#### enregistrement de la configuration d'une localisation trouvé

In [35]:
index_geo = int( input("Numéro de localisation à paramétrer : ") )

set_config( "lat", geo_match[index_geo][2] )
set_config( "lon", geo_match[index_geo][3] )
    
write_config_file( config )

Numéro de localisation à paramétrer : 0


### Appel API météo actuelle

In [58]:
# Endpoint current weather data
# Documentation : https://openweathermap.org/current

url_api = "http://api.openweathermap.org/data/2.5/weather?units=metric&lang=fr&appid=" + get_config("api_key")
url_api += "&lat=" + str( get_config("lat") )
url_api += "&lon=" + str( get_config("lon") )

# Executer la requete
req = requests.get( url_api )
api_data = req.json()

#print(api_data)

meteo = {}
meteo['meteo'] = api_data["weather"][0]["description"]
meteo['icon'] = api_data["weather"][0]["icon"]
meteo['lieu'] = api_data["name"]
meteo['coord_lon'] = api_data["coord"]["lon"]
meteo['coord_lat'] = api_data["coord"]["lat"]
meteo['vent_kph'] = api_data["wind"]["speed"] * 3.6
meteo['vent_dir'] = api_data["wind"]["deg"]
meteo['temp_ressentie'] = api_data["main"]["feels_like"]
meteo['temperature'] = api_data["main"]["temp"]
meteo['humidite'] = api_data["main"]["humidity"]
meteo['pression'] = api_data["main"]["pressure"]
meteo['sunrise'] = api_data["sys"]["sunrise"]
meteo['sunset'] = api_data["sys"]["sunset"]

print(meteo)

{'meteo': 'légère pluie', 'icon': '10n', 'lieu': 'Saint-Étienne', 'coord_lon': 4.3873, 'coord_lat': 45.4401, 'vent_kph': 18.503999999999998, 'vent_dir': 190, 'temp_ressentie': 16.47, 'temperature': 16.94, 'humidite': 68, 'pression': 1012, 'sunrise': 1666332370, 'sunset': 1666370863}


### Appel API prévisions météo

In [61]:
# Endpoint 5 day / 3 hour forecast data
# Documentation : https://openweathermap.org/forecast5

url_api = "http://api.openweathermap.org/data/2.5/forecast?units=metric&lang=fr&appid=" + get_config("api_key")
url_api += "&lat=" + str( get_config("lat") )
url_api += "&lon=" + str( get_config("lon") )

# Executer la requete
req = requests.get( url_api )
api_data = req.json()

print(api_data)



{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1666321200, 'main': {'temp': 16.94, 'feels_like': 16.47, 'temp_min': 15.1, 'temp_max': 16.94, 'pressure': 1012, 'sea_level': 1012, 'grnd_level': 951, 'humidity': 68, 'temp_kf': 1.84}, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'pluie modérée', 'icon': '10n'}], 'clouds': {'all': 100}, 'wind': {'speed': 4.22, 'deg': 179, 'gust': 12.64}, 'visibility': 3947, 'pop': 0.99, 'rain': {'3h': 10.17}, 'sys': {'pod': 'n'}, 'dt_txt': '2022-10-21 03:00:00'}, {'dt': 1666332000, 'main': {'temp': 15.86, 'feels_like': 15.49, 'temp_min': 13.71, 'temp_max': 15.86, 'pressure': 1012, 'sea_level': 1012, 'grnd_level': 952, 'humidity': 76, 'temp_kf': 2.15}, 'weather': [{'id': 502, 'main': 'Rain', 'description': 'forte pluie', 'icon': '10n'}], 'clouds': {'all': 100}, 'wind': {'speed': 2.17, 'deg': 175, 'gust': 3.03}, 'visibility': 10000, 'pop': 0.99, 'rain': {'3h': 16.97}, 'sys': {'pod': 'n'}, 'dt_txt': '2022-10-21 06:00:00'}, {'dt': 1666342800,

In [76]:
#for key in api_data:
#    print(key, api_data[key])
#    print()

#print(api_data['list'])

meteo_prev = []

for index, prev in enumerate( api_data['list'] ):
#    print(index, prev, type(prev), "\n")

    prevision = {}
    prevision['timestamp'] = prev['dt']
    prevision['date'] = prev['dt_txt']
    if prev['sys']['pod'] == "n":
        prevision['jour_nuit'] = "nuit"
    elif prev['sys']['pod'] == "d":
        prevision['jour_nuit'] = "jour"
    else:
        prevision['jour_nuit'] = ""
    prevision['temperature'] = round( prev['main']['temp'], 1 )
    prevision['temp_ressentie'] = round( prev['main']['feels_like'], 1 )
    prevision['temp_max'] = round( prev['main']['temp_max'], 1 )
    prevision['temp_min'] = round( prev['main']['temp_min'], 1 )
    prevision['pression'] = prev['main']['grnd_level']
    prevision['pression_mer'] = prev['main']['sea_level']
    prevision['humidite'] = prev['main']['humidity']
    prevision['meteo'] = prev['weather'][0]['description']
    prevision['meteo_code'] = prev['weather'][0]['main']
    prevision['meteo_icon'] = prev['weather'][0]['icon']
    prevision['nuages'] = prev['clouds']['all']
    prevision['vent_kph'] = round( prev["wind"]["speed"] * 3.6, 1 )
    prevision['vent_dir'] = prev["wind"]["deg"]
    prevision['vent_rafale'] = round( prev["wind"]["gust"] * 3.6, 1 )
    prevision['visibilite'] = prev["visibility"]
    prevision['pluie_probalite'] = int( prev["pop"] * 100 )
    if "rain" in prev:
        prevision['pluie_volume'] = round( prev["rain"]["3h"] )
    else:
        prevision['pluie_volume'] = 0
    if "snow" in prev:
        prevision['neige_volume'] = prev["snow"]["3h"]
    else:
        prevision['neige_volume'] = 0
    
    print(prevision, "\n")
    

{'timestamp': 1666321200, 'date': '2022-10-21 03:00:00', 'jour_nuit': 'nuit', 'temperature': 16.9, 'temp_ressentie': 16.5, 'temp_max': 16.9, 'temp_min': 15.1, 'pression': 951, 'pression_mer': 1012, 'humidite': 68, 'meteo': 'pluie modérée', 'meteo_code': 'Rain', 'meteo_icon': '10n', 'nuages': 100, 'vent_kph': 15.2, 'vent_dir': 179, 'vent_rafale': 45.5, 'visibilite': 3947, 'pluie_probalite': 99, 'pluie_volume': 10, 'neige_volume': 0} 

{'timestamp': 1666332000, 'date': '2022-10-21 06:00:00', 'jour_nuit': 'nuit', 'temperature': 15.9, 'temp_ressentie': 15.5, 'temp_max': 15.9, 'temp_min': 13.7, 'pression': 952, 'pression_mer': 1012, 'humidite': 76, 'meteo': 'forte pluie', 'meteo_code': 'Rain', 'meteo_icon': '10n', 'nuages': 100, 'vent_kph': 7.8, 'vent_dir': 175, 'vent_rafale': 10.9, 'visibilite': 10000, 'pluie_probalite': 99, 'pluie_volume': 17, 'neige_volume': 0} 

{'timestamp': 1666342800, 'date': '2022-10-21 09:00:00', 'jour_nuit': 'jour', 'temperature': 17.0, 'temp_ressentie': 16.8, 'te